In [1]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from pyspark.sql.functions import date_format
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


In [2]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "10g")
    .getOrCreate()
)

22/10/07 16:49:40 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/07 16:49:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 16:49:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 16:49:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Read in data from ETL.py file
%run '../scripts/outlier.py' '../scripts/paths.json'

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/07 16:49:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/07 16:50:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/07 16:51:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:

tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

In [5]:
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)

In [6]:
tagged_merchants_sdf.show(5)

+-------------------+--------------------+
|tagged_merchant_abn|            category|
+-------------------+--------------------+
|        10023283211|           Furniture|
|        10142254217|         Electronics|
|        10165489824|        Toys and DIY|
|        10187291046|        Toys and DIY|
|        10192359162|Books, Stationary...|
+-------------------+--------------------+
only showing top 5 rows



In [7]:
internal4.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
INNER JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")

joint = joint.drop('tagged_merchant_abn')

In [8]:
joint.count()

10109371

In [9]:
joint.createOrReplaceTempView("group")

a = spark.sql(""" 

SELECT *, (take_rate/100)*dollar_value AS percent
FROM group
""")

In [10]:
# Extracting the year, month, day from the timestamp
from pyspark.sql.functions import year, month

a = a.withColumn('Year', year(a.order_datetime))
a = a.withColumn('Month',month(a.order_datetime))


In [11]:
a.show(5)

+--------------------+------------+--------------------+---------+--------------+--------------------+--------------------+-----+------+------------------+------------------+--------------------+--------------+-------+-----------+-------+---------+----------+----------+---------+---------+-------------------+----------------+-----------+-------------+-------------+----------+---------------+---------------+---------------+--------------------------+--------------------------+--------------------+-------------------+----+-----+
|       merchant_name|merchant_abn|          categories|take_rate|revenue_levels|                name|             address|state|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id| suburb|postcodes|      long|       lat|  int_sa2| SA2_code|           SA2_name|income_2018-2019|total_males|total_females|total_persons|state_code|     state_name|population_2020|population_2021|fraud_probability_consumer|fraud_probabil

In [12]:
a = a.drop('merchant_abn', 'categories','name', 'address', 'trans_merchant_abn', 'order_id','order_datetime', 'consumer_id','int_sa2',
'SA2_name','state_code','state_name','population_2020', 'population_2021')

In [13]:
 
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
a.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in a.columns]
   ).show()

+-------------+---------+--------------+-----+------+------------+-------+------+---------+----+---+--------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------+-------+----+-----+
|merchant_name|take_rate|revenue_levels|state|gender|dollar_value|user_id|suburb|postcodes|long|lat|SA2_code|income_2018-2019|total_males|total_females|total_persons|fraud_probability_consumer|fraud_probability_merchant|category|percent|Year|Month|
+-------------+---------+--------------+-----+------+------------+-------+------+---------+----+---+--------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------+-------+----+-----+
|            0|        0|             0|    0|     0|           0|      0|     0|        0|   0|  0|       0|               0|          0|            0|            0|                         0|                         0|       0|      0|   0|    0|
+---

In [14]:
a.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- suburb: string (nullable = true)
 |-- postcodes: string (nullable = true)
 |-- long: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- SA2_code: long (nullable = true)
 |-- income_2018-2019: long (nullable = true)
 |-- total_males: long (nullable = true)
 |-- total_females: long (nullable = true)
 |-- total_persons: long (nullable = true)
 |-- fraud_probability_consumer: double (nullable = false)
 |-- fraud_probability_merchant: double (nullable = false)
 |-- category: string (nullable = true)
 |-- percent: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



In [15]:
a.createOrReplaceTempView("agg")

male = spark.sql(""" 

SELECT CONCAT(merchant_name, SA2_code, Year, Month) AS m_name, COUNT(gender) as males
FROM agg
WHERE gender = 'Male'
GROUP BY merchant_name, SA2_code, Year, Month
""")

male.show(5)

female = spark.sql(""" 

SELECT CONCAT(merchant_name, SA2_code, Year, Month) AS f_name, COUNT(gender) as females
FROM agg
WHERE gender = 'Female'
GROUP BY merchant_name, SA2_code, Year, Month
""")
female.show(5)

+--------------------+-----+
|              m_name|males|
+--------------------+-----+
|Semper Tellus PC1...|    2|
|Est Nunc Consulti...|   11|
|Ipsum Primis In I...|    2|
|Euismod In LLC601...|    1|
|Leo In Consulting...|    2|
+--------------------+-----+
only showing top 5 rows



+--------------------+-------+
|              f_name|females|
+--------------------+-------+
|Quis Tristique Lt...|      1|
|Nunc In Industrie...|      2|
|Nunc Sit LLC10902...|      2|
|Leo In Consulting...|      5|
|Risus Donec Assoc...|      1|
+--------------------+-------+
only showing top 5 rows



In [16]:
a.show(2)

+--------------------+---------+--------------+-----+------+-----------------+-------+-------+---------+----------+----------+---------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------------------+------------------+----+-----+
|       merchant_name|take_rate|revenue_levels|state|gender|     dollar_value|user_id| suburb|postcodes|      long|       lat| SA2_code|income_2018-2019|total_males|total_females|total_persons|fraud_probability_consumer|fraud_probability_merchant|            category|           percent|Year|Month|
+--------------------+---------+--------------+-----+------+-----------------+-------+-------+---------+----------+----------+---------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------------------+------------------+----+-----+
|Egestas Nunc Asso...|     6.58|             a|  NSW|  Male|11.28829564583802|   3698|JESMOND|     2299

In [17]:
a.createOrReplaceTempView("agg")

temp = spark.sql(""" 

SELECT merchant_name, COUNT(DISTINCT user_id) AS no_of_customers, SA2_code, Year, Month, SUM(dollar_value - percent) AS total_earnings,
    CONCAT(merchant_name, SA2_code, Year, Month) AS join_col
FROM agg 
GROUP BY merchant_name, SA2_code, Year, Month
""")

temp.show()


+--------------------+---------------+---------+----+-----+------------------+--------------------+
|       merchant_name|no_of_customers| SA2_code|Year|Month|    total_earnings|            join_col|
+--------------------+---------------+---------+----+-----+------------------+--------------------+
|Metus Sit Amet In...|              1|407021150|2021|    8| 66.38711669098895|Metus Sit Amet In...|
|     Ut Nisi Limited|              1|307031184|2021|    8| 89.47004686919264|Ut Nisi Limited30...|
|Dolor Dolor Indus...|              1|209041224|2021|    7| 32.32942469964551|Dolor Dolor Indus...|
|Ut Molestie Found...|              1|206041124|2021|    8| 327.0497650599944|Ut Molestie Found...|
|     Vivamus Sit LLC|              1|307031184|2021|    8| 244.8254892010557|Vivamus Sit LLC30...|
|Euismod Et Institute|              7|210021235|2021|    8| 194.5570510377872|Euismod Et Instit...|
|   Leo In Consulting|             20|210021235|2021|    8|1172.0943954300897|Leo In Consulting...|


In [18]:
temp.createOrReplaceTempView("gender_join")
male.createOrReplaceTempView("m")
female.createOrReplaceTempView("f")

temp2 = spark.sql(""" 

SELECT *
FROM gender_join
INNER JOIN m
ON gender_join.join_col = m.m_name
""")

temp2.createOrReplaceTempView("temp2")

temp3 = spark.sql(""" 

SELECT *
FROM temp2
INNER JOIN f
ON temp2.join_col = f.f_name
""")

temp3.limit(5)

merchant_name,no_of_customers,SA2_code,Year,Month,total_earnings,join_col,m_name,males,f_name,females
A Aliquet Ltd,2,401021010,2021,4,670.4857203238805,A Aliquet Ltd4010...,A Aliquet Ltd4010...,1,A Aliquet Ltd4010...,1
A Aliquet Ltd,2,603011065,2021,12,346.5799250465364,A Aliquet Ltd6030...,A Aliquet Ltd6030...,1,A Aliquet Ltd6030...,1
A Arcu Industries,2,124011453,2021,8,203.5414742977921,A Arcu Industries...,A Arcu Industries...,1,A Arcu Industries...,1
A Arcu Industries,2,211051282,2022,3,655.1195924003883,A Arcu Industries...,A Arcu Industries...,1,A Arcu Industries...,1
A Arcu Industries,2,214021379,2021,7,451.0067711100705,A Arcu Industries...,A Arcu Industries...,1,A Arcu Industries...,1


In [19]:
a = a.withColumnRenamed('income_2018-2019',

    'income_2018_2019'    
)

a = a.withColumn('income_per_persons',
    (F.col('income_2018_2019')/F.col('total_persons'))
)


In [20]:
a.show(1)

+--------------------+---------+--------------+-----+------+-----------------+-------+-------+---------+----------+----------+---------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------------------+------------------+----+-----+------------------+
|       merchant_name|take_rate|revenue_levels|state|gender|     dollar_value|user_id| suburb|postcodes|      long|       lat| SA2_code|income_2018_2019|total_males|total_females|total_persons|fraud_probability_consumer|fraud_probability_merchant|            category|           percent|Year|Month|income_per_persons|
+--------------------+---------+--------------+-----+------+-----------------+-------+-------+---------+----------+----------+---------+----------------+-----------+-------------+-------------+--------------------------+--------------------------+--------------------+------------------+----+-----+------------------+
|Egestas Nunc Asso...|     6.58|             a

In [21]:
a.createOrReplaceTempView("features")

e = spark.sql(""" 

SELECT merchant_name AS drop_name, FIRST(take_rate) AS take_rate, FIRST(revenue_levels) AS revenue_levels, FIRST(category) AS category,
    FIRST(total_males) AS males_in_SA2, FIRST(total_females) AS females_in_SA2, FIRST(income_per_persons) AS income_per_person
FROM features
GROUP BY merchant_name
""")

e.show(2)

+---------------+---------+--------------+--------------------+------------+--------------+------------------+
|      drop_name|take_rate|revenue_levels|            category|males_in_SA2|females_in_SA2| income_per_person|
+---------------+---------+--------------+--------------------+------------+--------------+------------------+
|   A Associates|     4.95|             b|Books, Stationary...|        9762|         10846|22526.523772559674|
|A Felis Company|     4.32|             b|Books, Stationary...|        1080|          1051| 33927.61168708765|
+---------------+---------+--------------+--------------------+------------+--------------+------------------+
only showing top 2 rows



In [22]:
temp3.createOrReplaceTempView("edit")
e.createOrReplaceTempView("rates")

temp4 = spark.sql(""" 

SELECT *
FROM edit
INNER JOIN rates
ON edit.merchant_name = rates.drop_name
""")

train = temp4.drop('m_name', 'f_name', 'drop_name','join_col')

train.limit(5)

merchant_name,no_of_customers,SA2_code,Year,Month,total_earnings,males,females,take_rate,revenue_levels,category,males_in_SA2,females_in_SA2,income_per_person
A Aliquet Ltd,2,312021351,2021,6,298.8537411028336,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Aliquet Ltd,2,401021010,2021,4,670.4857203238805,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Aliquet Ltd,2,603011065,2021,12,346.5799250465364,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Arcu Industries,2,124011453,2021,8,203.5414742977921,1,1,3.0,c,Furniture,4821,4683,25816.03452631579
A Arcu Industries,2,211051282,2022,3,655.1195924003883,1,1,3.0,c,Furniture,4821,4683,25816.03452631579


In [23]:
train.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_customers: long (nullable = false)
 |-- SA2_code: long (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: long (nullable = false)
 |-- females: long (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- category: string (nullable = true)
 |-- males_in_SA2: long (nullable = true)
 |-- females_in_SA2: long (nullable = true)
 |-- income_per_person: double (nullable = true)



In [24]:
train_projection = train.select("merchant_name", "SA2_code", "Year", "Month", 'no_of_customers')
train_projection.limit(5)

merchant_name,SA2_code,Year,Month,no_of_customers
A Aliquet Ltd,401021010,2021,4,2
A Aliquet Ltd,603011065,2021,12,2
A Arcu Industries,124011453,2021,8,2
A Arcu Industries,211051282,2022,3,2
A Arcu Industries,214021379,2021,7,2


In [25]:
train_projection.count()

891622

In [26]:
train_projection = train_projection.withColumn("prev_year", \
              when(train_projection["Month"] == 1, train_projection['Year'] - 1).otherwise(train_projection['Year']))
train_projection = train_projection.withColumn("prev_month", \
              when(train_projection["Month"] == 1, 12).otherwise(train_projection['Month'] - 1))
train_projection = train_projection.drop("Year", "Month")
train_projection = train_projection.withColumnRenamed("no_of_customers", "future_customers") \
                            .withColumnRenamed("merchant_name", "p_merchant_name") \
                            .withColumnRenamed("SA2_code", "p_SA2_code")
train_projection.limit(5)

p_merchant_name,p_SA2_code,future_customers,prev_year,prev_month
A Aliquet Ltd,401021010,2,2021,3
A Aliquet Ltd,603011065,2,2021,11
A Arcu Industries,124011453,2,2021,7
A Arcu Industries,211051282,2,2022,2
A Arcu Industries,214021379,2,2021,6


In [27]:
train_projection.count()

891622

In [28]:
final_data = train.join(train_projection, (train.merchant_name == train_projection.p_merchant_name) & 
                           (train.SA2_code == train_projection.p_SA2_code) & 
                           (train.Year == train_projection.prev_year) & 
                           (train.Month == train_projection.prev_month), how = 'inner')

final_data = final_data.drop("p_merchant_name", "p_SA2_code","prev_year", "prev_month")
final_data.limit(5)

merchant_name,no_of_customers,SA2_code,Year,Month,total_earnings,males,females,take_rate,revenue_levels,category,males_in_SA2,females_in_SA2,income_per_person,future_customers
A Auctor Non Corp...,3,202031033,2021,11,209.32764854012208,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,6
A Auctor Non Corp...,3,205021082,2022,7,262.10605847773724,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,3
A Auctor Non Corp...,2,205031087,2021,9,218.6890344781834,1,1,5.58,a,Furniture,2067,2014,22634.72370679088,3
A Auctor Non Corp...,3,210021235,2022,9,152.99495946306212,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,3
A Auctor Non Corp...,4,211051282,2021,10,298.778609499402,2,2,5.58,a,Furniture,2067,2014,22634.72370679088,5


In [29]:
final_data = final_data.withColumn('Year',

    F.col('Year').cast('STRING')

)

final_data = final_data.withColumn('Month',

    F.col('Month').cast('STRING')

)

final_data = final_data.withColumn('SA2_code',

    F.col('SA2_code').cast('STRING')

)

field = ['future_customers','no_of_customers' ,'males', 'females', 'males_in_SA2', 'females_in_SA2']

for col in field:
    final_data = final_data.withColumn(col,

    F.col(col).cast('INT')

)

In [30]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'SA2_code', 'Year', 'Month', 'revenue_levels','category'],
outputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'], handleInvalid="keep")

indexd_data = indexer.fit(final_data).transform(final_data)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'],
outputCols = ['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec','males_in_SA2','females_in_SA2','no_of_customers' ,'income_per_person','take_rate', 'total_earnings'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

In [31]:
# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "future_customers",
    "label"
)

In [32]:
# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [33]:
# Split the data into training and validation sets (30% held out for testing)

trainingData, testData = outdata1.randomSplit([0.7, 0.3], seed = 20)

In [34]:
trainingData.count(), testData.count()

22/10/07 17:05:10 WARN DAGScheduler: Broadcasting large task binary with size 1447.5 KiB


22/10/07 17:06:35 WARN DAGScheduler: Broadcasting large task binary with size 1447.5 KiB


(240534, 103485)

In [35]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")


# Train model.  
model = rf.fit(trainingData)

# Make predictions.
predictions_validation = model.transform(testData)

22/10/07 17:07:57 WARN DAGScheduler: Broadcasting large task binary with size 1447.4 KiB


22/10/07 17:07:58 WARN DAGScheduler: Broadcasting large task binary with size 1447.5 KiB


22/10/07 17:08:00 WARN DAGScheduler: Broadcasting large task binary with size 1451.5 KiB


22/10/07 17:08:02 WARN DAGScheduler: Broadcasting large task binary with size 1564.2 KiB


22/10/07 17:08:09 WARN DAGScheduler: Broadcasting large task binary with size 1625.7 KiB


22/10/07 17:08:12 WARN DAGScheduler: Broadcasting large task binary with size 1746.6 KiB


22/10/07 17:08:16 WARN DAGScheduler: Broadcasting large task binary with size 1985.6 KiB


22/10/07 17:08:21 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


In [36]:
# Evaluate the validation set 

predictions_validation.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error

evaluator_train_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse_train = evaluator_train_rmse.evaluate(predictions_validation)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_train)

evaluator_train_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae_train = evaluator_train_mae.evaluate(predictions_validation)
print("Mean Absolutee Error (MAE) on train data = %g" % mae_train)

22/10/07 17:09:46 WARN DAGScheduler: Broadcasting large task binary with size 1461.7 KiB


+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 3.995871420804393|    4|(2085,[209,1093,2...|
| 4.105122129941754|    6|(2085,[209,988,20...|
| 4.105122129941754|    4|(2085,[209,976,20...|
| 4.416280714838842|    5|(2085,[209,989,20...|
|4.5805087950826735|    3|(2085,[209,1039,2...|
+------------------+-----+--------------------+
only showing top 5 rows



22/10/07 17:11:10 WARN DAGScheduler: Broadcasting large task binary with size 1452.7 KiB


22/10/07 17:11:12 WARN DAGScheduler: Broadcasting large task binary with size 1453.8 KiB
Root Mean Squared Error (RMSE) on train data = 2.95724


22/10/07 17:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1452.7 KiB


22/10/07 17:12:33 WARN DAGScheduler: Broadcasting large task binary with size 1453.8 KiB
Mean Absolutee Error (MAE) on train data = 2.0985


In [37]:
def ExtractFeatureImportance(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
  
#ExtractFeatureImportance(model.stages[-1].featureImportances, dataset, "features")
dataset_fi = ExtractFeatureImportance(model.featureImportances, predictions_validation, "features")
dataset_fi = spark.createDataFrame(dataset_fi)
display(dataset_fi)

idx,name,score
2081,no_of_customers,0.7728913228246239
2084,total_earnings,0.12018637343537428
2082,income_per_person,0.01819117331506711
0,merchant_name_vec...,0.010906124748100212
2,merchant_name_vec...,0.009304060195289543
2079,males_in_SA2,0.00766412157028098
2066,Month_vec_12,0.006715216883223568
2083,take_rate,0.006413892437917029
1,merchant_name_vec...,0.005814341361763084
2080,females_in_SA2,0.00550944865680363


## Future predictions

In [38]:
latest_year = train.select(max('Year')).collect()[0][0]
agg_month_1 = train.filter(train.Year == latest_year)
latest_month = agg_month_1.select(max('Month')).collect()[0][0]
predicting_data = agg_month_1.filter(train.Month == latest_month)
predicting_data = predicting_data.withColumn("future_customers", lit(0))
predicting_data.limit(5)

AnalysisException: Column 'r' does not exist. Did you mean one of the following? [edit.Year, edit.Month, edit.males, edit.females, edit.SA2_code, rates.category, rates.take_rate, rates.males_in_SA2, edit.merchant_name, edit.total_earnings, rates.females_in_SA2, edit.no_of_customers, rates.revenue_levels, rates.income_per_person];
'Project ['r]
+- Project [merchant_name#193, no_of_customers#2750L, SA2_code#608L, Year#1749, Month#1785, total_earnings#2751, males#2487L, females#2539L, take_rate#3842, revenue_levels#3844, category#3846, males_in_SA2#3848L, females_in_SA2#3850L, income_per_person#3852]
   +- Project [merchant_name#193, no_of_customers#2750L, SA2_code#608L, Year#1749, Month#1785, total_earnings#2751, join_col#2752, m_name#2486, males#2487L, f_name#2538, females#2539L, drop_name#3840, take_rate#3842, revenue_levels#3844, category#3846, males_in_SA2#3848L, females_in_SA2#3850L, income_per_person#3852]
      +- Join Inner, (merchant_name#193 = drop_name#3840)
         :- Subque

In [ ]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'SA2_code', 'Year', 'Month', 'revenue_levels','category'],
outputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'], handleInvalid="keep")

indexd_data = indexer.fit(predicting_data).transform(predicting_data)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'],
outputCols = ['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec','males_in_SA2','females_in_SA2', 'income_per_person', 'no_of_customers','take_rate', 'total_earnings'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "future_customers",
    "label"
)


# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [ ]:
predictions_test = model.transform(outdata1)

In [ ]:
predictions_test.show(1)

22/10/05 12:35:11 WARN DAGScheduler: Broadcasting large task binary with size 1196.9 KiB
+--------------------+---------------+---------+----+-----+------------------+-----+-------+---------+--------------+---------+------------+--------------+-----------------+-----+-----------------+------------+--------+---------+------------------+------------+------------------+------------------+-------------+-------------+------------------+-------------+--------------------+--------------------+------------------+
|       merchant_name|no_of_customers| SA2_code|Year|Month|    total_earnings|males|females|take_rate|revenue_levels| category|males_in_SA2|females_in_SA2|income_per_person|label|merchant_name_num|SA2_code_num|Year_num|Month_num|revenue_levels_num|category_num| merchant_name_vec|      SA2_code_vec|     Year_vec|    Month_vec|revenue_levels_vec| category_vec|            features|     indexedFeatures|        prediction|
+--------------------+---------------+---------+----+-----+--------

In [ ]:
predictions_test.createOrReplaceTempView("preds")

pred = spark.sql(""" 

SELECT merchant_name, ROUND(SUM(prediction)) AS total_future_customers
FROM preds
GROUP BY merchant_name

""")

pred.limit(5)

merchant_name,total_future_customers
Dictum Mi Incorpo...,8.0
Dictum Mi Limited,214.0
Donec Luctus Indu...,56.0
Elit Sed Consequa...,182.0
Hendrerit Consect...,36.0


In [ ]:
pred.count()

1381

In [ ]:
pred_df = pred.toPandas()

In [ ]:
pred_df.to_csv("../data/curated/customers.csv")